In [1]:
file_path = "20220510_083559"

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
# import import_mimic as impt

import tensorflow as tf
from class_DeepLongitudinal import Model_Longitudinal_Attention

/home/ugrads/t/troyalty18/miniconda3/envs/ddh/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ugrads/t/troyalty18/miniconda3/envs/ddh/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ugrads/t/troyalty18/miniconda3/envs/ddh/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ugrads/

In [3]:
_EPSILON = 1e-08

import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os

from sklearn.model_selection import train_test_split

# import import_mimic_new as impt


from class_DeepLongitudinal import Model_Longitudinal_Attention

from utils_eval             import c_index, brier_score
from utils_log              import save_logging, load_logging
from utils_helper           import f_get_minibatch, f_get_boosted_trainset

In [28]:
mask1.shape

(31544, 1, 2441)

In [4]:
# import import_mimic as impt
# (x_dim, x_dim_cont, x_dim_bin), (data, time, label), (mask1, mask2, mask3), (data_mi) = impt.import_dataset(norm_mode = 'standard', out="old")

In [5]:
mask1.shape

(31544, 1, 120)

In [7]:
data_mode                   = 'PBC2' 
seed                        = 1234
(x_dim, x_dim_cont, x_dim_bin), (data, time, label), (mask1, mask2, mask3), (data_mi) = impt.import_dataset(norm_mode = 'standard')

_, num_Event, num_Category  = np.shape(mask1)  # dim of mask3: [subj, Num_Event, Num_Category]
max_length                  = np.shape(data)[1]

(tr_data,te_data, tr_data_mi, te_data_mi, tr_time,te_time, tr_label,te_label, 
 tr_mask1,te_mask1, tr_mask2,te_mask2, tr_mask3,te_mask3) = train_test_split(data, data_mi, time, label, mask1, mask2, mask3, test_size=0.2, random_state=seed) 

(tr_data,va_data, tr_data_mi, va_data_mi, tr_time,va_time, tr_label,va_label, 
 tr_mask1,va_mask1, tr_mask2,va_mask2, tr_mask3,va_mask3) = train_test_split(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3, test_size=0.125, random_state=seed) 

In [8]:
print(tr_data.shape, va_data.shape, te_data.shape)

(22080, 1069, 18) (3155, 1069, 18) (6309, 1069, 18)


In [9]:
def _f_get_pred(sess, model, data, data_mi, pred_horizon):
    '''
        predictions based on the prediction time.
        create new_data and new_mask2 that are available previous or equal to the prediction time (no future measurements are used)
    '''
    new_data    = np.zeros(np.shape(data))
    new_data_mi = np.zeros(np.shape(data_mi))

    meas_time = np.concatenate([np.zeros([np.shape(data)[0], 1]), np.cumsum(data[:, :, 0], axis=1)[:, :-1]], axis=1)

    for i in range(np.shape(data)[0]):
        last_meas = np.sum(meas_time[i, :] <= pred_horizon)

        new_data[i, :last_meas, :]    = data[i, :last_meas, :]
        new_data_mi[i, :last_meas, :] = data_mi[i, :last_meas, :]

    return model.predict(new_data, new_data_mi)


def f_get_risk_predictions(sess, model, data_, data_mi_, pred_time, eval_time):
    
    pred = _f_get_pred(sess, model, data_[[0]], data_mi_[[0]], 0)
    _, num_Event, num_Category = np.shape(pred)
       
    risk_all = {}
    for k in range(num_Event):
        risk_all[k] = np.zeros([np.shape(data_)[0], len(pred_time), len(eval_time)])
            
    for p, p_time in enumerate(pred_time):
        ### PREDICTION
        pred_horizon = int(p_time)
        pred = _f_get_pred(sess, model, data_, data_mi_, pred_horizon)
        #print(pred.shape)
        print("Pred ==============================================")
        print(pred)


        for t, t_time in enumerate(eval_time):
            eval_horizon = int(t_time) + pred_horizon #if eval_horizon >= num_Category, output the maximum...

            # calculate F(t | x, Y, t >= t_M) = \sum_{t_M <= \tau < t} P(\tau | x, Y, \tau > t_M)
            risk = np.sum(pred[:,:,pred_horizon:(eval_horizon+1)], axis=2) #risk score until eval_time
            risk = risk / (np.sum(np.sum(pred[:,:,pred_horizon:], axis=2), axis=1, keepdims=True) +_EPSILON) #conditioniong on t > t_pred
            
            for k in range(num_Event):
                risk_all[k][:, p, t] = risk[:, k]
                
    return pred, risk_all

In [11]:
burn_in_mode                = 'ON' #{'ON', 'OFF'}
boost_mode                  = 'OFF' #'ON' #{'ON', 'OFF'}

##### HYPER-PARAMETERS
# TODO: Change to 32
new_parser = {'mb_size': 4,

             'iteration_burn_in': 3000,
             #'iteration': 25000,
              'iteration': 10000,

             'keep_prob': 0.6,
             'lr_train': 1e-4,

             'h_dim_RNN': 100,
             'h_dim_FC' : 100,
             'num_layers_RNN':2,
             'num_layers_ATT':2,
             'num_layers_CS' :2,

             'RNN_type':'LSTM', #{'LSTM', 'GRU'}

             'FC_active_fn' : tf.nn.relu,
             'RNN_active_fn': tf.nn.tanh,

            'reg_W'         : 1e-5,
            'reg_W_out'     : 0.,

             'alpha' :1.0,
             'beta'  :0.1,
             'gamma' :1.0
}


# INPUT DIMENSIONS
input_dims                  = { 'x_dim'         : x_dim,
                                'x_dim_cont'    : x_dim_cont,
                                'x_dim_bin'     : x_dim_bin,
                                'num_Event'     : num_Event,
                                'num_Category'  : num_Category,
                                'max_length'    : max_length }
print(input_dims)

# NETWORK HYPER-PARMETERS
network_settings            = { 'h_dim_RNN'         : new_parser['h_dim_RNN'],
                                'h_dim_FC'          : new_parser['h_dim_FC'],
                                'num_layers_RNN'    : new_parser['num_layers_RNN'],
                                'num_layers_ATT'    : new_parser['num_layers_ATT'],
                                'num_layers_CS'     : new_parser['num_layers_CS'],
                                'RNN_type'          : new_parser['RNN_type'],
                                'FC_active_fn'      : new_parser['FC_active_fn'],
                                'RNN_active_fn'     : new_parser['RNN_active_fn'],
                                'initial_W'         : tf.contrib.layers.xavier_initializer(),

                                'reg_W'             : new_parser['reg_W'],
                                'reg_W_out'         : new_parser['reg_W_out']
                                 }

{'x_dim': 18, 'x_dim_cont': 16, 'x_dim_bin': 1, 'num_Event': 1, 'num_Category': 144, 'max_length': 1069}


In [12]:
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = Model_Longitudinal_Attention(sess, "Dyanmic-DeepHit", input_dims, network_settings)
saver = tf.train.Saver()
saver.restore(sess, file_path + '/model')

INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from 20220510_083559/model


2022-05-12 13:38:04.785927: W tensorflow/core/framework/op_kernel.cc:1401] OP_REQUIRES failed at save_restore_v2_ops.cc:184 : Not found: Key Dyanmic-DeepHit/Dyanmic-DeepHit/rnn/multi_rnn_cell/cell_0/lstm_cell/bias/Adam not found in checkpoint


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key Dyanmic-DeepHit/Dyanmic-DeepHit/rnn/multi_rnn_cell/cell_0/lstm_cell/bias/Adam not found in checkpoint
	 [[node save/RestoreV2 (defined at /tmp/ipykernel_445469/3450577834.py:7) ]]

Caused by op 'save/RestoreV2', defined at:
  File "/home/ugrads/t/troyalty18/miniconda3/envs/ddh/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ugrads/t/troyalty18/miniconda3/envs/ddh/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ugrads/t/troyalty18/miniconda3/envs/ddh/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ugrads/t/troyalty18/miniconda3/envs/ddh/lib/python3.7/site-packages/traitlets/config/

In [ ]:
_EPSILON = 1e-08

pred, risk_all = f_get_risk_predictions(sess, model, va_data, va_data_mi, pred_time, eval_time)

for p, p_time in enumerate(pred_time):
    pred_horizon = int(p_time)
    val_result1 = np.zeros([num_Event, len(eval_time)])

    for t, t_time in enumerate(eval_time):                
        eval_horizon = int(t_time) + pred_horizon
        for k in range(num_Event):
            val_result1[k, t] = c_index(risk_all[k][:, p, t], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

    if p == 0:
        val_final1 = val_result1
    else:
        val_final1 = np.append(val_final1, val_result1, axis=0)

tmp_valid = np.mean(val_final1)
print('val_loss: {:.4f}'.format(tmp_valid))

In [ ]:
preds_train = model.predict(tr_data, tr_data_mi)
preds_val = model.predict(va_data, va_data_mi)
preds_test = model.predict(te_data, te_data_mi)

preds_train.shape, preds_val.shape, preds_test.shape

In [ ]:
pred_time = [30, 60, 90]
eval_time = [30, 60, 90]
saver.restore(sess, file_path + '/model')


pred_test, risk_all = f_get_risk_predictions(sess, model, te_data, te_data_mi, pred_time, eval_time)

for p, p_time in enumerate(pred_time):
    pred_horizon = int(p_time)
    result1, result2 = np.zeros([num_Event, len(eval_time)]), np.zeros([num_Event, len(eval_time)])

    for t, t_time in enumerate(eval_time):                
        eval_horizon = int(t_time) + pred_horizon
        for k in range(num_Event):
            result1[k, t] = c_index(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
            result2[k, t] = brier_score(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
    
    if p == 0:
        final1, final2 = result1, result2
    else:
        final1, final2 = np.append(final1, result1, axis=0), np.append(final2, result2, axis=0)
        
        
row_header = []
for p_time in pred_time:
    for t in range(num_Event):
        row_header.append('pred_time {}: event_{}'.format(p_time,k+1))
            
col_header = []
for t_time in eval_time:
    col_header.append('eval_time {}'.format(t_time))

# c-index result
df1 = pd.DataFrame(final1, index = row_header, columns=col_header)

# brier-score result
df2 = pd.DataFrame(final2, index = row_header, columns=col_header)

### PRINT RESULTS
print('========================================================')
print('--------------------------------------------------------')
print('- C-INDEX: ')
print(df1)
print('--------------------------------------------------------')

print('- BRIER-SCORE: ')
print(df2)
print('========================================================')

# Flagging function

In [ ]:
N_OUTPUTS = 120

def get_flag_df(preds_df):
    for_flagging = pd.DataFrame()
    preds_df = preds_test.reshape(-1, N_OUTPUTS)
    preds_df.shape

    n_subjects = preds_df.shape[0]
    
    arr = np.array([])
    for i in range(n_subjects):
        new = np.repeat(i, N_OUTPUTS)
        arr = np.concatenate((arr, new), axis=None)
        
    for_flagging['id'] = arr
    for_flagging['hazard'] = preds_test.reshape(n_subjects*N_OUTPUTS)
    
    l_array = np.array([]) 
    for i, label in enumerate(te_label.reshape(te_label.shape[0])):
        if label == 0:
            l_array = np.concatenate([l_array, np.repeat(0, N_OUTPUTS)])
        else:
            curr = np.repeat(0, N_OUTPUTS)
            curr[te_time_ceil[i]] = 1
            l_array = np.concatenate([l_array, curr])
    for_flagging['label'] = l_array
    
    for_flagging['Y'] = np.repeat(1, for_flagging.shape[0])
    for_flagging.to_csv(file_path + '/for_flagging.csv', index=False)
    return for_flagging

In [ ]:
get_flag_df(preds_test)